In [ ]:
# !pip install pydub

In [1]:
import os
import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
TAVILY_API_KEY = keyring.get_password('tavily', 'key_for_windows')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [2]:
# set web search and scaping tools
from typing import Annotated, List

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool

tavily_tool = TavilySearchResults(max_results=5)

@tool
def scape_webpage(urls: List[str]) -> str:
    """Use requests and bs4 to scape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return "\n\n".join(
        [
            f"\n{doc.page_content}\n"
            for doc in docs
        ]
    )

In [ ]:
# set TTS using OpenAI TTS API
from langchain_core.tools import tool
from openai import OpenAI
from pathlib import Path

@tool
def text_to_speech(text: str, voice: str, output_filename: str) -> str:
    """
    Converts text to speech using OpenAI's TTs API, saving the audio file locally with a dynamic filename.
    
    Args:
        text (str): the text to convert to speech.
        voice (str): the voice model to use for speech sythesis. Options include:
                    - 'alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer'.
        output_filenmae (str): the name for the output audio file, including file extension (e.g., 'output.mp3')
        
    Returns:
        str: the path to the saved audio file
    """
    client = OpenAI()
    speech_file_path = Path(output_filename)
    
    response = client.audio.speech.create(model='tts-1-hd', voice=voice, input=text)
    
    # save audio file
    # response.stream_to_file(speech_file_path)      # deprecated
    with open(speech_file_path, 'wb') as audio_file:
        for chunk in response.iter_content(chunk_size=1024):
            audio_file.write(chunk)
    
    return str(speech_file_path)